# Textbook Chatbot (Team 3)

**Purpose**: This chatbot is as an educational tool that's built to answer questions related to the textbook, [Software Engineering Body of Knowledge (SWEBOK)](https://www.computer.org/education/bodies-of-knowledge/software-engineering). The chatbot was built by team 3 for [CSE 6550: Software Engineering Concepts](https://catalog.csusb.edu/coursesaz/cse/)

**Objective**: In this notebook, we will demonstrate how the chatbot uses retrieval augemented generation (RAG) to answer questions using the SWEBOK textbook as the primary data source.

**Prerequisites**:
Github, Docker, Mamba, Python, Jupyter Notebook

**Resources**:
[![GitHub](https://img.shields.io/badge/GitHub-black?style=flat&logo=github&logoColor=white)](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team3) 
[![Wiki](https://img.shields.io/badge/Wiki-blue?style=flat&logo=wikipedia&logoColor=white)](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team3/wiki)

## Table of Contents
1. [Setup](#1.-Setup)
   - [Creating Virtual Environment](#1.1-Creating-Virtual-Environment)
   - [Importing Dependencies](#1.2-Importing-dependencies)
3. [Building the Chatbot](#2.-Building-the-Chatbot)
   - [Document loading](#2.1-Document-loading)
   - [Embeddings](#2.2-Embeddings)
   - [LLM setup](#2.3-LLM-setup)
   - [Mistral loader](#2.4-Mistral-loader)
4. [Improving the Chatbot with inference](#3.-Improving-the-Chatbot-with-inference)
   - [Helpful functions](#3.1-Helpful-functions)
   - [Prompt engineering](#3.2-Prompt-engineering)
5. [Testing the Chatbot](#4.-Testing-the-Chatbot)
6. [Conclusion](#5.-Conclusion)

## 1. Setup

To ensure compatibility, it is necessary to verify the Python version installed on your system. This project requires Python 3.10 or higher. Follow these steps to check and prepare your environment:

Steps to Verify Python Version:

- Check Installed Version:
Open your terminal or command prompt and execute the following command:

- For windows or Linux OS use command
    ```python --version```

- For Macos Os use command
    ```!python3 --version```

Dependency Requirements:
- Python must already be installed on your system.
- Python version of 3.10 or higher is mandatory for this project to function correctly.

- If Python is not installed, download and install the latest version of Python from the official Python website.
https://www.python.org/downloads/

In [4]:
!python3 --version

Python 3.12.0


### 1.1 Creating Virtual Environment
This code is setting up a virtual environment for Python. Here's what it does in simple terms:

- Install Required Tools:
    - It makes sure necessary Python packages (`ipykernel` and `virtualenv`) are installed.
    - These are tools needed for creating and managing the virtual environment.

- Create a Virtual Environment:
    - It creates a virtual environment named `myenv`.
    - A virtual environment is like a separate workspace where you can install Python packages without affecting the global system settings.

In [5]:
!python3 -m venv team3_env

In [7]:
!team3_env/bin/pip install ipykernel -q


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [8]:
!team3_env/bin/python3 -m ipykernel install --user --name=team3_env

Installed kernelspec team3_env in /Users/purav/Library/Jupyter/kernels/team3_env


#### Switch Kernel and Verify:

1. Switch the Kernel
- In the Jupyter Notebook interface, go to the menu bar
- Select `Kernel > Change Kernel`.
- A list of available kernels will appear.
- Switch to ```team3_env```

2. Verify the Kernel
- After switching the kernel, you can verify it by running the following commands in a new code cell:

In [1]:
import sys
if "team3_env" not in sys.executable:
    print("Switch kernel to team3_env! You will not be able to proceed unless you switch the kernel to team3_env.")
else:
    print("Using team3_env as kernel!")

Using team3_env as kernel!


#### Update `pip` if needed

In [2]:
print("Updating pip")
%pip install --upgrade pip -q

Updating pip
Note: you may need to restart the kernel to use updated packages.


### 1.2 Importing dependencies

This cell installs essential packages for the chatbot and data processing.

In [7]:
import sys
import os

def install_dependencies():
    """
    Installs required Python libraries using pip with the -q (quiet) flag.
    """
    print("Installing dependencies. This can take up to 3 minutes.")

    # Upgrade pip
    try:
        print("Upgrading pip...")
        !{sys.executable} -m pip install --upgrade pip -q
    except Exception as e:
        print(f"Error upgrading pip: {e}")
        return

    # Define required libraries
    libraries = [
        "faiss-cpu", "huggingface_hub", "ipykernel", "jupyter", "langchain",
        "langchain-community", "langchain-huggingface", "langchain-mistralai",
        "python-dotenv", "pypdf", "requests", "sentence-transformers", "tiktoken"
    ]

    # Install dependencies with -q flag for quiet output
    try:
        print("Installing required libraries...")
        libraries_str = " ".join(libraries)
        !{sys.executable} -m pip install -q {libraries_str}
        print("Dependencies installed successfully.")
    except Exception as e:
        print(f"Error occurred during dependency installation: {e}")

# Call the function to install dependencies
install_dependencies()

Installing dependencies. This can take up to 3 minutes.
Upgrading pip...
Installing required libraries...
Dependencies installed successfully.


## 2. Building the Chatbot

### 2.1 Document loading

- **Purpose**: 
The code loads documents from a specified directory to build the data corpus for the chatbot

- **Input**:
The input refers to the documents loaded from the directory specified by document_path, which will be used to process user queries related to the chatbot's knowledge base.

- **Output**:
The output is the collection of documents loaded from the directory into the `documents` variable, which will be used for further processing in the chatbot.

- **Processing**:
    - The code loads documents from the specified directory into the `documents` variable, creating a corpus for the chatbot to use in responding to queries.
    - The primary data source used in this project is [Software Engineering Body of Knowledge (SWEBOK)](https://www.computer.org/education/bodies-of-knowledge/software-engineering).

In [8]:
# Importing required modules
import os
import sys
import requests
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Raw URLs and their local paths
files = [
    {
        "url": "https://raw.githubusercontent.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team3/49c98e61fc7219c152fb102aa14f630c005150e3/data/swebok/faiss_indexes/collection/index.pkl",
        "path": "faiss_indexes/collection/index.pkl"
    },
    {
        "url": "https://raw.githubusercontent.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team3/49c98e61fc7219c152fb102aa14f630c005150e3/data/swebok/faiss_indexes/collection/index.faiss",
        "path": "faiss_indexes/collection/index.faiss"
    },
    {
        "url": "https://raw.githubusercontent.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team3/49c98e61fc7219c152fb102aa14f630c005150e3/data/swebok/textbook.pdf",
        "path": "textbook.pdf"
    }
]

# Create directories
os.makedirs('faiss_indexes/collection', exist_ok=True)

# Download each file
for file in files:
    try:
        print(f"Downloading {file['path']}...")
        response = requests.get(file['url'])
        response.raise_for_status()
        
        with open(file['path'], 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded {file['path']}")
        
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {file['path']}: {e}")

print("Download process completed!")

Successfully downloaded faiss_indexes/collection/index.pkl
Successfully downloaded faiss_indexes/collection/index.faiss
Successfully downloaded textbook.pdf
Download process completed!


In [10]:
# Importing required modules
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("Loading document. This can take up to 1 minute.")

# Adds the parent directory to the Python path for module imports
sys.path.append(os.path.dirname(os.getcwd())) 

# Defines corpus source and paths for documents and FAISS indexes
corpus_source = "swebok"  # Sets SWEBOK as corpus
document_path = os.getcwd()
persist_directory = os.path.join(document_path, "faiss_indexes")  # Directory for storing FAISS indexes

def load_documents_from_directory(document_path: str, chunk_size: int = 2048, chunk_overlap: int = 200):
    """
    Load PDF documents from a directory and split them into chunks.
    
    Args:
        document_path (str): Path to the directory containing PDF files.
        chunk_size (int): Size of each text chunk (default: 2048).
        chunk_overlap (int): Overlap between chunks (default: 200).
        
    Returns:
        List of document chunks.
    """
    try:
        print(f"Loading documents from {document_path}...")

        # Check if the document path exists
        if not os.path.exists(document_path):
            raise FileNotFoundError(f"Directory not found: {document_path}")

        # Load PDF documents from the specified directory
        loader = PyPDFDirectoryLoader(document_path)
        documents = loader.load()

        # Create a text splitter using tiktoken encoder
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=chunk_size, 
            chunk_overlap=chunk_overlap
        )

        # Split the documents into chunks
        return text_splitter.split_documents(documents)

    except Exception as e:
        print(f"An error occurred while loading documents: {e}")
        return []

# Load all documents from the defined directory
documents = load_documents_from_directory(document_path)

# Print the number of documents loaded
print(f"Number of documents loaded: {len(documents)}")

Loading document. This can take up to 1 minute.
Loading documents from /Users/purav/Documents/CSUSB/CSE_6550_SWE_Concepts/csusb_fall2024_cse6550_team3/jupyter...
Number of documents loaded: 412


### 2.2 Embeddings

- **Purpose**:
Download and initialize the embedding model from HuggingFace to generate vector embeddings for text.

- **Input**:
The input is the model name (`"Alibaba-NLP/gte-large-en-v1.5"`) which is used to fetch the embedding model from HuggingFace.

- **Output**:
The output is the `EMBEDDING_FUNCTION`, which is an instance of the `HuggingFaceEmbeddings` class, ready to generate embeddings for text using the specified model.

- **Processing**:
    - We have retrieved the textbook, we need to create vector embeddings for it
    - We will use [FAISS](https://python.langchain.com/docs/integrations/vectorstores/faiss/) as our vector database and [Alibaba-NLP/gte-large-en-v1.5](https://huggingface.co/Alibaba-NLP/gte-large-en-v1.5) as our embedding model

In [13]:
print("Creating/loading the embeddings this will take a couple of minutes...")
# Download the embedding model from HuggingFace
from langchain_huggingface import HuggingFaceEmbeddings
EMBEDDING_MODEL_NAME = "Alibaba-NLP/gte-large-en-v1.5"
EMBEDDING_FUNCTION = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={'trust_remote_code': True})
print("Loaded embedding model successfully")

Creating/loading the embeddings this will take a couple of minutes...
Loaded embedding model successfully


#### Loading Vector store

- **Purpose**:
To create or load FAISS vector embeddings for the documents, allowing for efficient retrieval during chatbot interactions.

- **Input**:
The input is the `documents` (loaded documents to be embedded) and `persist_directory` (the directory to store or retrieve the FAISS index).

- **Output**:
The output is `faiss_store`, which is a FAISS vector store containing the document embeddings for efficient search and retrieval.

- **Processing**:
    - The `load_or_create_faiss_vector_store` function processes the documents by either creating new FAISS embeddings or loading existing ones from the specified directory (`persist_directory`), enabling fast document retrieval based on vector similarity.

In [16]:
print("Please wait... This step can take a long time when performed for the first time")
# Using pre-built load_or_create_faiss_vector_store function to create or load FAISS embeddings

from langchain_community.vectorstores import FAISS

def load_or_create_faiss_vector_store(
	documents,
	persist_directory,
	collection_name="collection"
):
	"""
	Load an existing FAISS vector store or create a new one if it doesn't exist.
	Args:
			documents: List of documents to be indexed.
			collection_name (str): Name of the collection.
			persist_directory (str): Directory to save/load the FAISS index.
	Returns:
			FAISS vector store object.
	"""
	index_path = os.path.join(persist_directory, f'{collection_name}')
	if os.path.exists(index_path):
		# Load existing FAISS index
		print(f"Loading existing FAISS vector store from {index_path}...\n")
		faiss_store = FAISS.load_local(
			index_path, 
			embeddings=EMBEDDING_FUNCTION, 
			allow_dangerous_deserialization=True
		)
	else:
		# Create new FAISS index
		print(f"Creating new FAISS vector store in {index_path}...\n")
		faiss_store = FAISS.from_documents(
			documents, 
			embedding=EMBEDDING_FUNCTION
		)
		faiss_store.save_local(index_path)
	return faiss_store

faiss_store = load_or_create_faiss_vector_store(documents, persist_directory)
print("FAISS vector store loaded.")

Please wait... This step can take a long time when performed for the first time
Loading existing FAISS vector store from /Users/purav/Documents/CSUSB/CSE_6550_SWE_Concepts/csusb_fall2024_cse6550_team3/jupyter/faiss_indexes/collection...

FAISS vector store loaded.


### 2.3 LLM setup

### Environment Variables

<ul>
<li><strong>Purpose</strong>:<br>
<ul>
    <li>To load environment variables from a <code>.env</code> file, retrieve the Mistral API key, and ensure the key is available for further usage in the application.</li>
</ul>
</li>
<br>
<li><strong>Input</strong>:
<ul>
    <li><code>.env</code> file containing environment variables (like <code>MISTRAL_API_KEY</code>)</li>
    <li><code>api_key</code>: A string variable that may hold the Mistral API key (if manually provided)</li>
</ul>
</li>
<br>
<li><strong>Output</strong>:<br>
<ul>
    <li>Prints "Environment variables successfully setup" if successful, or raises an error if the <code>MISTRAL_API_KEY</code> is not found</li>
</ul>
</li>
<br>
<li><strong>Processing</strong>:<br>
We have to setup environment variables that will contain our API keys:
<ul>
    <li>If you have already created a <code>.env</code> file and added the <code>MISTRAL_API_KEY</code> you do not have to do anything</li>
    <li>If not, then you can add your API key below. Get an API key <a href="https://console.mistral.ai/api-keys/">here</a></li>
</ul>
</li>
</ul>

In [18]:
from dotenv import load_dotenv
load_dotenv(override=True)

api_key = "" # add your Mistral API key here if needed
if api_key == "":
    api_key = os.getenv("MISTRAL_API_KEY")
elif not api_key:
	raise ValueError("You have to enter your MISTRAL_API_KEY or add it to a .env file")
print("Environment variables succesfully setup")

Environment variables succesfully setup


### 2.4 Mistral loader

**Purpose**:
To load the Mistral AI model (in this case, "open-mistral-7b") using the `ChatMistralAI` class from `langchain_mistralai`, and configure it with necessary parameters (such as temperature, max tokens, and top-p) for generating responses.

**Input**:
- `model_name`: The name of the pre-trained model to be used, here set as "open-mistral-7b".
- `api_key`: The Mistral API key used for authenticating the model access.

**Output**:
- The model is loaded and ready to be used for generating responses.
- Prints "Successfully loaded Mistral 7B" upon successful loading of the model.

**Processing**:
We will be using [Mistral 7B](https://mistral.ai/news/announcing-mistral-7b/) as our primary large language model. This will combined with our retriever to create our RAG application.

In [21]:
from langchain_mistralai import ChatMistralAI

# Load and configure the Mistral AI LLM.
MODEL_NAME = "mistral-large-2411"
llm = ChatMistralAI(model=MODEL_NAME, mistral_api_key=api_key, temperature=0, max_tokens=256)
print("Successfully loaded Mistral LLM")

Successfully loaded Mistral LLM


## 3. Improving the Chatbot

### 3.1 Helpful functions

#### Similarity search
<ul>
<li><strong>Purpose</strong>:<br>
Retrieve and filter the top k most similar documents from the FAISS vector store based on a question.</li>
<br>
<li><strong>Input</strong>:
<ul>
    <li><code>question</code>: The user's query</li>
    <li><code>vector_store</code>: FAISS vector store</li>
    <li><code>k</code>: Number of similar documents to return</li>
    <li><code>distance_threshold</code>: Score threshold for filtering</li>
</ul>
</li>
<br>
<li><strong>Output</strong>:<br>
Filtered list of documents that are most similar to the question</li>
<br>
<li><strong>Processing</strong>:<br>
Perform a similarity search in the vector store, filter documents based on the score threshold, return the relevant documents</li>
</ul>

In [25]:
# Get top k most similar documents using FAISS vector store.
def similarity_search(question, vector_store, k, distance_threshold = 420.0):
	retrieved_docs = vector_store.similarity_search_with_score(question, k=k)
	filtered_docs = [doc for doc, score in retrieved_docs if score <= distance_threshold]
	return filtered_docs
print("Creared similarity search function.")

Creared similarity search function.


#### Chat completion

<ul>
<li><strong>Purpose</strong>:<br>
Generate responses to a user's question using the RAG system by combining relevant documents and the LLM</li>
<br>
<li><strong>Input</strong>:
<ul>
    <li><code>question</code>: The user's query</li>
    <li><code>prompt</code>: The system's prompt format</li>
    <li><code>llm</code>: The language model to generate the response</li>
</ul>
</li>
<br>
<li><strong>Output</strong>:<br>
Streamed response chunks as an answer, enriched with context from the relevant documents</li>
<br>
<li><strong>Processing</strong>:<br>
Retrieve the top k relevant documents using <code>similarity_search</code>, format the context from the documents and the user query, use the LLM to generate a response, streaming chunks of the answer</li>
</ul>

In [26]:
# Uses the RAG system to answer the user's questions
def chat_completion(question, prompt, llm):
    top_k = 10 # The maximum number of documents that similarity search will return
    
    relevant_docs = similarity_search(question, faiss_store, top_k) # Get relevant documents
    
    context = "\n\n".join([doc.page_content for doc in relevant_docs]) # Format retrived documents
    messages = prompt.format_messages(input=question, context=context) 
    
    # Stream response
    full_response = {"answer": "", "context": relevant_docs}
    for chunk in llm.stream(messages):
        full_response["answer"] += chunk.content
        yield (chunk.content)
print("Created chat completion function.")

Created chat completion function.


#### Interactive chatbot interface
<ul>
<li><strong>Purpose</strong>:<br>
Provide an interactive interface where the user can input a question (or prompt) and get a response from the chatbot by invoking the RAG system</li>
<br>
<li><strong>Input</strong>:<br>
User-provided prompt (query) through the <code>prompt_input</code> widget</li>
<br>
<li><strong>Output</strong>:<br>
Display the chatbot's response, streamed in chunks, in the output widget</li>
<br>
<li><strong>Processing</strong>:<br>
<ul>
    <li>Create Input/Output Widgets: <code>prompt_input</code> for user input, <code>submit_button</code> for triggering the action, and <code>output</code> to display the response</li>
    <li>Button Click Action: When the button is clicked, it triggers the <code>on_submit</code> function</li>
    <li>Response Generation: The function uses the <code>chat_completion</code> to generate a response based on the user query. It then streams and displays the response in the <code>output</code> widget</li>
</ul>
</li>
</ul>

In [37]:
import ipywidgets as widgets
from IPython.display import display

# Prompt widget
prompt_input = widgets.Text(
    placeholder='Enter your prompt here...',
    description='Prompt:',
    layout=widgets.Layout(width='500px')
)
# Sumbit button
submit_button = widgets.Button(
    description='Submit',
    button_style='primary'
)
output = widgets.Output()
def on_submit(b):
    with output:
        output.clear_output()
        user_prompt = prompt_input.value
        if not user_prompt:
            user_prompt = "Who is Hironori Washizaki?"
        print(f"\nPrompt: {user_prompt}\n")
        print(f"------------------------------\n")
        # Stream the response
        for response_chunk in chat_completion(user_prompt, prompt, llm):
            print(response_chunk, end='', flush=True)

submit_button.on_click(on_submit)
print("Created interactive chatbot interface.")

Created interactive chatbot interface.


### 3.2 Prompt engineering
<ul>
<li><strong>Purpose</strong>:<br>
Define the system behavior for the chatbot and format the prompt template for interacting with the user and providing responses based on the context.</li>
<br>
<li><strong>Input</strong>:
<ul>
    <li>The <code>system_prompt</code> specifies the chatbot's instructions for how to respond</li>
    <li>The prompt template defines how the question and context are formatted for the chatbot</li>
</ul>
</li>
<br>
<li><strong>Output</strong>:<br>
A formatted prompt template (<code>prompt</code>) that combines the system instructions and user input (question and context) for processing by the LLM</li>
<br>
<li><strong>Processing</strong>:
<ul>
<li>System Behavior Definition: The <code>system_prompt</code> sets rules for how the chatbot should answer questions, handle uncertainty, and identify itself</li>
<li>Prompt Template Creation: The <code>ChatPromptTemplate</code> is created with a combination of the system prompt and the user's question/context format, ready to be used for generating the response</li>
</ul>
</li>
</ul>

In [38]:
from langchain_core.prompts import ChatPromptTemplate

# The system prompt will be used as a framework drive the LLM responses
SYSTEM_PROMPT = """
You are a chatbot that answers the question inside the <question_start><question_end> tags.
Use the context provided to answer the question.
If the context is not relavent answer the question if it's related to software engineering.
The context is present within the <context_start> and <context_end> tags.
"""

# Setting up a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "<question_start>{input}<question_end>\n\n<context_start>{context}<context_end>"),
])

print("Created prompt template")

Created prompt template


# 4. Testing the Chatbot

<ul>
<li><strong>Purpose</strong>:<br>
Render interactive widgets for user input, a submit button, and output display in the notebook</li>
<br>
<li><strong>Input</strong>:
<ul>
<li><code>prompt_input</code>: User's query</li>
<li><code>submit_button</code>: Button to trigger response generation</li>
<li><code>output</code>: Area to display the response</li>
</ul>
</li>
<br>
<li><strong>Output</strong>:<br>
Displays input field, submit button, and output area for chatbot interaction</li>
<br>
<li><strong>Processing</strong>:
<ul>
<li>User enters a query and clicks the button</li>
<li>The chatbot processes the query and displays the response in the output area</li>
</ul>
</li>
</ul>

In [40]:
display(prompt_input, submit_button, output)

Text(value='MVC', description='Prompt:', layout=Layout(width='500px'), placeholder='Enter your prompt here...'…

Button(button_style='primary', description='Submit', style=ButtonStyle())

Output(outputs=({'name': 'stdout', 'text': '\nPrompt: MVC\n\n------------------------------\n\nMVC stands for …

# 5. Conclusion

Recap:
- Developed a chatbot using the RAG system, which retrieves relevant documents and generates responses based on the context provided
- Integrated widgets in Jupyter Notebook for interactive user input and response display
- Configured the chatbot with natural language processing models like Mistral AI and vector-based document retrieval using FAISS

Resources:
- The Textbook Chatbot project was built by Team 3 for [CSE 6550: Software Engineering Concepts](https://catalog.csusb.edu/coursesaz/cse/) offered at CSUSB

[![GitHub](https://img.shields.io/badge/GitHub-black?style=flat&logo=github&logoColor=white)](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team3) 
[![Wiki](https://img.shields.io/badge/Wiki-blue?style=flat&logo=wikipedia&logoColor=white)](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team3/wiki)